### Progetto 8b
Luca Maria Lauricella

Valerio Marini

Repository del progetto:
https://github.com/lauriluca99/TGW-3D.jl

Documentazione del progetto:
https://lauriluca99.github.io/TGW-3D.jl

In [2]:
using BenchmarkTools
using SparseArrays
using Distributed
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

    function spatial_arrangement_1(
            V::Points,
            copEV::ChainOp,
            copFE::ChainOp, 
            [multiproc::Bool=false])
			
Si occupa del processo di frammentazione delle facce per l'utilizzo del planar arrangement.	
Richiama le funzioni `frag_face` e `merge_vertices' per ritornare i nuovi vertici, lati e facce.
#### Argomenti addizionali:
- `multiproc::Bool`: Esegue la computazione in modalità parallela. Di Default a `false`.

In [3]:
function spatial_arrangement_1(
    V::Lar.Points,
    copEV::Lar.ChainOp,
    copFE::Lar.ChainOp, multiproc::Bool=false)

# spaceindex computation
FV = Lar.compute_FV( copEV, copFE )
model = (convert(Lar.Points,V'), FV)
sp_idx = Lar.spaceindex(model)

# initializations
fs_num = size(copFE, 1)
rV = Array{Float64,2}(undef,0,3)
rEV = SparseArrays.spzeros(Int8,0,0)
rFE = SparseArrays.spzeros(Int8,0,0)

# multiprocessing of face fragmentation
if (multiproc == true)
    in_chan = Distributed.RemoteChannel(()->Channel{Int64}(100))
    out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(100))
    @async begin
        for sigma in 1:fs_num
            
            put!(in_chan, sigma)
            @show sigma
        end
        for p in Distributed.workers()
           @show p
           put!(in_chan, -1)
        end
    end
    for p in Distributed.workers()
        
        @async Base.remote_do(frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        
    end
    
    for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
       

    end
else
# sequential (iterative) processing of face fragmentation
    for sigma in 1:fs_num
        #print(sigma, "/", fs_num, "\r")
        nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
        #nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
        nV = convert(Lar.Points, nV)
        a,b,c = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
        rV=a;  rEV=b;  rFE=c
    end
end
# merging of close vertices, edges and faces (3D congruence)
rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
return rV, rEV, rFE
end

spatial_arrangement_1 (generic function with 2 methods)

In [4]:
V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
]

EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
])

FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
])

6×12 SparseMatrixCSC{Int8, Int64} with 24 stored entries:
  1   1   1  -1   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -1  -1  -1   1
 -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅
  ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅
  ⋅   ⋅  -1   ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅
  ⋅   ⋅   ⋅   1  -1   ⋅   ⋅   1   ⋅   ⋅   ⋅  -1

In [5]:
@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  3.850 ms (54201 allocations: 2.72 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 9, 1, 2, 10, 2, 3, 11, 3  …  9, 5, 6, 10, 6, 7, 11, 7, 8, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  1, 1, -1, 1, 1, -1, 1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  2, 6, 3, 6, 3, 4, 4, 5, 5, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, 1, 1, 1, 1, 1, -1, 1, 1, -1  …  -1, -1, -1, -1, 1, -1, 1, -1, 1, 1], 6, 12))

In [6]:
@code_warntype spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

MethodInstance for spatial_arrangement_1(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Bool)
  from spatial_arrangement_1(V::Matrix, copEV::SparseMatrixCSC{Int8, Int64}, copFE::SparseMatrixCSC{Int8, Int64}, multiproc::Bool) in Main at In[3]:1
Arguments
  #self#::Core.Const(spatial_arrangement_1)
  V::Matrix{Float64}
  copEV::SparseMatrixCSC{Int8, Int64}
  copFE::SparseMatrixCSC{Int8, Int64}
  multiproc::Bool
Locals
  @_6::Any
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  #2::var"#2#6"
  #1::var"#1#5"
  rFE::Any
  rEV::Any
  rV::Any
  out_chan::Any
  in_chan::Any
  fs_num::Int64
  sp_idx::Any
  model::Tuple{Any, Any}
  FV::Any
  #3::var"#3#7"{_A, Int64} where _A
  task@_22::Task
  p::Int64
  #4::var"#4#8"{Matrix{Float64}, _A, _B, _C, Int64} where {_A, _B, _C}
  task@_25::Task
  @_26::Any
  sigma@_27::Int64
  @_28::Any
  @_29::Any
  c::Any
  b::Any
  a::Any
  nFE::A

# Ottimizzazione

In [7]:
function spatial_arrangement_12(
    V::Lar.Points,
    copEV::Lar.ChainOp,
    copFE::Lar.ChainOp, multiproc::Bool=false)

# spaceindex computation
FV = Lar.compute_FV( copEV, copFE )
model = (convert(Lar.Points,V'), FV)
sp_idx = Lar.spaceindex(model)

# initializations
fs_num = size(copFE, 1)
rV = Array{Float64,2}(undef,0,3)
rEV = SparseArrays.spzeros(Int8,0,0)
rFE = SparseArrays.spzeros(Int8,0,0)
# sequential (iterative) processing of face fragmentation
     for sigma in 1:fs_num
        #print(sigma, "/", fs_num, "\r")
        nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
        #nV, nEV, nFE = Lar.fragface(V, copEV, copFE, sp_idx, sigma)
        nV = convert(Lar.Points, nV)
        rV, rEV, rFE = Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
        #rV=a;  rEV=b;  rFE=c
    end
# merging of close vertices, edges and faces (3D congruence)
rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
return rV, rEV, rFE
end

spatial_arrangement_12 (generic function with 2 methods)

In [8]:
@btime spatial_arrangement_12(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE))

  3.974 ms (54201 allocations: 2.72 MiB)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], sparse([1, 4, 9, 1, 2, 10, 2, 3, 11, 3  …  9, 5, 6, 10, 6, 7, 11, 7, 8, 12], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  5, 6, 6, 6, 7, 7, 7, 8, 8, 8], Int8[-1, -1, -1, 1, -1, -1, 1, -1, -1, 1  …  1, 1, -1, 1, 1, -1, 1, 1, 1, 1], 12, 8), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  2, 6, 3, 6, 3, 4, 4, 5, 5, 6], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  8, 8, 9, 9, 10, 10, 11, 11, 12, 12], Int8[1, 1, 1, 1, 1, 1, -1, 1, 1, -1  …  -1, -1, -1, -1, 1, -1, 1, -1, 1, 1], 6, 12))